In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.sparse.linalg
from PIL import Image

plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
def plot(x1, x2):
  print("x1 mean: ", np.mean(x1, axis=(0,1)))
  print("x2 mean: ", np.mean(x2, axis=(0,1)))
  fig, ax = plt.subplots(1,2)
  plt.setp(ax, xticks=[], yticks=[])
  plt.tight_layout()
  ax[0].imshow(x1)
  ax[1].imshow(x2)
  plt.show()

In [ ]:
def normalize_staining(x, beta=0.15, alpha=1, light_intensity=255):
  """
  Normalize the staining of H&E histology slides.
  
  This function normalizes the staining of H&E histoloy slides.
  
  References:
    - Macenko, Marc, et al. "A method for normalizing histology slides for
    quantitative analysis." Biomedical Imaging: From Nano to Macro, 2009.
    ISBI'09. IEEE International Symposium on. IEEE, 2009.
      - http://wwwx.cs.unc.edu/~mn/sites/default/files/macenko2009.pdf
    - https://github.com/mitkovetta/staining-normalization/blob/master/normalizeStaining.m
  """
  # Setup.
  x = np.asarray(x)
  h, w, c = x.shape
  x = x.reshape(-1, c).astype(np.float64)  # shape (H*W, C)
  
  # Reference stain vectors and stain saturations.  We will normalize all slides
  # to these references.  To create these, grab the stain vectors and stain
  # saturations from a desirable slide.
  ## Values in reference implementation for use with eigendecomposition approach.
#   stain_ref = np.array([0.5626, 0.2159, 0.7201, 0.8012, 0.4062, 0.5581]).reshape(3,2)
#   max_sat_ref = np.array([1.9705, 1.0308]).reshape(2,1)
  ## Values for use with SVD approach.  These were computed by (1) running the
  ## the eigendecomposition approach to normalize an image, (2) running the
  ## SVD approach on the normalized image, and (3) recording the stain vectors
  ## and max saturations for this (ideal) normalized image.
#   stain_ref = np.array([0.20730702, 0.56170196, 0.80308092, 0.72012455, 0.55864554, 0.4073224]).reshape(3,2)
#   max_sat_ref = np.array([0.99818645, 1.96029115]).reshape(2,1)
  # SVD w/ explicitly ordered H&E stains (BEST!)
#   stain_ref = np.array([0.5373251, 0.20661399, 0.73087638, 0.78939496, 0.42083424, 0.57807115]).reshape(3,2)
#   max_sat_ref = np.array([1.95105957, 0.98605565]).reshape(2,1)
  # SVD w/ explicitly ordered H&E stains & log10 OD
  stain_ref = np.array([0.54598845, 0.322116, 0.72385198, 0.76419107, 0.42182333, 0.55879629]).reshape(3,2)
  max_sat_ref = np.array([0.82791151, 0.61137274]).reshape(2,1)
  
  # SVD w/ beta=0.35
#   stain_ref = np.array([0.323427, 0.54875885, 0.77909905, 0.72734252, 0.53702853, 0.41211234]).reshape(3,2)
#   max_sat_ref = np.array([1.43153082, 1.73216312]).reshape(2,1)
  
  # Convert RGB to OD.
#   OD = -np.log((x+1)/light_intensity)  # shape (H*W, C)
  OD = -np.log10(x/light_intensity + 1e-8)
  
  # Remove data with OD intensity less than beta.
  # I.e. remove transparent pixels.
  # Note: This needs to be checked per channel, rather than
  # taking an average over all channels for a given pixel.
  #OD_thresh = OD[np.logical_not(np.any(OD < beta, 1)), :]
  OD_thresh = OD[np.all(OD >= beta, 1), :]  # shape (K, C)
  
  # Calculate eigenvectors.
#   eigvals, eigvecs = np.linalg.eig(np.cov(OD_thresh.T))  # np.cov results in inf/nans
  U, s, V = np.linalg.svd(OD_thresh, full_matrices=False)
  
  # Extract two largest eigenvectors.
  # Note: We swap the sign of the eigvecs here to be consistent
  # with other implementations.  Both +/- eigvecs are valid, with
  # the same eigenvalue, so this is okay.
#   top_eigvecs = eigvecs[:, np.argsort(eigvals)[-2:]] * -1
  top_eigvecs = V[0:2, :].T * -1  # shape (C, 2)
  
  # Project thresholded optical density values onto plane spanned by
  # 2 largest eigenvectors.
  proj = np.dot(OD_thresh, top_eigvecs)  # shape (K, 2)
  
  # Calculate angle of each point wrt the first plane direction.
  # Note: the parameters are `np.arctan2(y, x)`
  angles = np.arctan2(proj[:, 1], proj[:, 0])  # shape (K,)
  
  # Find robust extremes (a and 100-a percentiles) of the angle.
  min_angle = np.percentile(angles, alpha)
  max_angle = np.percentile(angles, 100-alpha)
  
  # Convert min/max vectors (extremes) back to OD space.
#   extreme_angles = np.array(
#     [np.cos(min_angle), np.cos(max_angle), np.sin(min_angle), np.sin(max_angle)]
#   ).reshape(2,2)
#   stains = np.dot(top_eigvecs, extreme_angles)  # shape (C, 2)
  min_vec = np.dot(top_eigvecs, np.array([np.cos(min_angle), np.sin(min_angle)]).reshape(2,1))
  max_vec = np.dot(top_eigvecs, np.array([np.cos(max_angle), np.sin(max_angle)]).reshape(2,1))
  
  # Merge vectors with hematoxylin first, and eosin second with a heuristic.
  if min_vec[0] > max_vec[0]:
    stains = np.hstack((min_vec, max_vec))
  else:
    stains = np.hstack((max_vec, min_vec))

  # Calculate saturations of each stain.
  # Note: Here, we solve
  #    OD = VS
  #     S = V^{-1}OD
  # where `OD` is the matrix of optical density values of our image,
  # `V` is the matrix of stain vectors, and `S` is the matrix of stain
  # saturations.  Since this is an overdetermined system, we use the
  # least squares solver, rather than a direct solve.
  sats, _, _, _ = np.linalg.lstsq(stains, OD.T)
  
  # Normalize stain saturations.
  max_sat = np.percentile(sats, 99, axis=1, keepdims=True)
  sats = sats / max_sat * max_sat_ref
  
  # Recreate image.
  # Note: If the image is immediately converted to uint8 with `.astype(np.uint8)`, it will
  # not return the correct values due to the initital values being outside of [0,255].
  # To fix this, we round to the nearest integer, and then clip to [0,255], which is the
  # same behavior as Matlab.
#   x_norm = np.exp(np.dot(-stain_ref, sats)) * light_intensity
  x_norm = 10**(np.dot(-stain_ref, sats)) * light_intensity - 1e-8
  x_norm = np.clip(np.round(x_norm), 0, 255).astype(np.uint8)
  x_norm = x_norm.T.reshape(h,w,c)
  
  # Debug.
#   print("OD shape: ", OD.shape)
#   print("OD_thresh shape: ", OD_thresh.shape)
#   print("eigvals: ", eigvals)
#   print("sorted eigvals: ", np.argsort(eigvals))
#   print("top_eigvecs shape: ", top_eigvecs.shape)
#   print("top_eigvecs: ", top_eigvecs)
#   print("top 2 eigval indices: ", np.argsort(eigvals)[-2:])
#   print("proj shape: ", proj.shape)
#   print("proj mean: ", np.mean(proj, axis=0))
#   print("angles shape: ", angles.shape)
#   print("angles mean: ", np.mean(angles))
#   print("min/max angles: ", min_angle, max_angle)
#   print("min_vec shape: ", min_vec.shape)
#   print("min_vec mean: ", np.mean(min_vec))
#   print("max_vec mean: ", np.mean(max_vec))
#   print("stains shape: ", stains.shape)
  print("stains: ", stains)
#   print("sats shape: ", sats.shape)
#   print("sats mean: ", np.mean(sats, axis=1))
#   print("max_sat shape: ", max_sat.shape)
  print("max_sat: ", max_sat)
#   print("x_norm shape: ", x_norm.shape)
#   print("x_norm mean: ", np.mean(x_norm, axis=(0,1)))
#   print("x_norm min: ", np.min(x_norm, axis=(0,1)))
#   print("x_norm max: ", np.max(x_norm, axis=(0,1)))
#   print(x_norm.dtype)
#   print()
# #   x = x.reshape(h,w,c).astype(np.uint8)
  
  return x_norm

In [ ]:
x1 = np.asarray(Image.open("staining-normalization/example1.tif"))
x2 = np.asarray(Image.open("staining-normalization/example2.tif"))
x3 = np.asarray(Image.open("literature/cancer_zoom.jpg"))
print(x1.dtype)
plot(x1, x2)
plot(x1, x3)

In [ ]:
x1_norm = normalize_staining(x1)
x2_norm = normalize_staining(x2)
x3_norm = normalize_staining(x3)
x1_norm2 = normalize_staining(x1_norm)
plot(x1_norm, x2_norm)
plot(x1_norm2, x3_norm)
#Image.fromarray(x1_norm).save("images/x1_norm.jpg")
# Note: x1 mean:  [ 190.95086861  143.9767971   169.32534027] for eigendecomposition approach
#       x1 mean:  [ 192.84288502  145.66594315  170.64605522] for SVD approach

In [ ]:
# Bootstrap SVD approach using x1_norm from eigendecomposition approach
# with `stains` and `max_sat` debugging turned on.
x1_norm = Image.open("images/x1_norm.jpg")
x1_norm2 = normalize_staining(x1_norm)
plot(x1_norm, x1_norm2)
# Note: x1 mean:  [ 190.95086861  143.9767971   169.32534027] for eigenvector approach

In [ ]:
# Check SVD approach during bootstrapping
x1_norm_b = normalize_staining(x1)
x2_norm_b = normalize_staining(x2)
x3_norm_b = normalize_staining(x3)
x1_norm2_b = normalize_staining(x1_norm)
plot(x1_norm_b, x2_norm_b)
plot(x1_norm2_b, x3_norm_b)

In [ ]:
# Compute the x and y coordinates for points on a sine curve
x = np.arange(0, 3 * np.pi, 0.1)
y = np.sin(x)

# Plot the points using matplotlib
plt.plot(x, y)
plt.show()  # You must call plt.show() to make graphics appear.

In [ ]:
x = np.random.randn(10, 3)
x[np.mean(x, 1) > 0.9, :].shape